In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from urllib.request import urlopen
import folium
import branca
import scipy as sp
import scipy.ndimage
import geojsoncontour
from math import radians
from folium import plugins
from scipy.interpolate import griddata
from sklearn.metrics.pairwise import haversine_distances
import folium
from sklearn.cluster import KMeans
from folium.plugins import MousePosition
import warnings
from pyecharts.charts import Pie, Bar, Map, WordCloud,Line,Grid,Scatter,Radar,Page 
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.globals import ThemeType
from pyecharts.charts import Bar, Line, Grid,Tab
from pyecharts.globals import ThemeType
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts.render import make_snapshot
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Grid, Tab
warnings.filterwarnings("ignore")
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.plotting import *
from bokeh.models import ColumnDataSource
from bokeh.models import Legend
from bokeh.plotting import figure, show, output_file
from bokeh.palettes import *
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.embed import components
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.embed import file_html
%matplotlib inline

In [2]:
df = pd.read_csv('Fire_Incidents.csv')
# select variables that we need
df1 = df[['Incident Number', 'Address', 'Incident Date','Primary Situation','point',"Suppression Personnel",'Action Taken Primary',
          'Item First Ignited','Ignition Cause', 'Alarm DtTm', 'Arrival DtTm',
          'Close DtTm', 'Area of Fire Origin']]

#clean data
df1.dropna(inplace = True)
df2 = df1[df1['Item First Ignited'] != "-"]
df3 = df2[df2['Action Taken Primary'] != "-"]
df4 = df3[df3['Area of Fire Origin'] != "-"] 

# select the date from 2003-01-01 to 2019-12-31
df4['Date'] = df4['Incident Date'].str.slice(0,10)
df5=df4[df4['Date']  <=  "2019-12-31"]

# split the longitudes and  latitude
a = df5.point.str.slice(7,-1)
longitudes = []
latitudes = []

for location in list(a):
    values = str(location).split()
    if len(values) == 1:
        longitude, latitude = None, None
    else:
        longitude, latitude = map(float, values)
    longitudes.append(longitude)
    latitudes.append(latitude)
    
df5['longitude'] = longitudes
df5['latitudes'] = latitudes

In [3]:
from ipywidgets import interact
import folium

def plot_fire_incidents(year):
    m = folium.Map(location=[37.7749,-122.4194], zoom_start=12, tiles='Stamen Terrain') 
    data = {
            'latitude': list(df5[df5['Date'].str.contains(year)]["latitudes"]),
            'longitude': list(df5[df5['Date'].str.contains(year)]["longitude"])
            }
    df = pd.DataFrame(data)
    for index, row in df.iterrows():
        folium.CircleMarker([row['latitude'], row['longitude']], radius=0.00001,
                            color='red', fill_color='red', fill_opacity=10).add_to(m)

    X = df5[df5['Date'].str.contains(year)][["latitudes","longitude"]]
    kmeans = KMeans(n_clusters=10, random_state=42)
    kmeans.fit(X)
    centroids = kmeans.cluster_centers_

    MousePosition(position='bottomleft', separator=' | ').add_to(m)
    folium.Marker(location=[centroids[0][0], centroids[0][1]],icon=folium.Icon(icon = "flag",color='blue')).add_to(m)
    folium.Marker(location=[centroids[6][0], centroids[6][1]],icon=folium.Icon(icon = "flag",color='blue')).add_to(m)
    folium.Marker(location=[centroids[8][0], centroids[8][1]],icon=folium.Icon(icon = "flag",color='blue')).add_to(m)
    folium.Marker(location=[centroids[3][0], centroids[3][1]],icon=folium.Icon(icon = "flag",color='blue'),popup='Centroids 1').add_to(m)

    folium.Marker(location=[centroids[2][0], centroids[2][1]],icon=folium.Icon(icon = "star",color='green')).add_to(m)
    folium.Marker(location=[centroids[9][0], centroids[9][1]],icon=folium.Icon(icon = "star",color='green')).add_to(m)
    folium.Marker(location=[centroids[1][0], centroids[1][1]],icon=folium.Icon(icon = "star",color='green')).add_to(m)

    folium.Marker(location=[centroids[5][0], centroids[5][1]],icon=folium.Icon(icon = "cloud",color='gray')).add_to(m)
    folium.Marker(location=[centroids[4][0], centroids[4][1]],icon=folium.Icon(icon = "cloud",color='gray')).add_to(m)

    folium.Marker(location=[centroids[7][0], centroids[7][1]],icon=folium.Icon(icon = "play",color='green')).add_to(m)
    
    return m

interact(plot_fire_incidents, year=list(df5['Date'].str.slice(0, 4).unique()))


interactive(children=(Dropdown(description='year', options=('2008', '2003', '2004', '2005', '2006', '2007', '2…

<function __main__.plot_fire_incidents(year)>